# Week 4 Presentation

In [2]:
import numpy as np  # useful for many scientific computing in Python

import pandas as pd # primary data structure library
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopandas
import geopandas as gpd
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt


import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values



#!conda install -c conda-forge folium=0.5.0 --yes
import folium
import shapely

from shapely.wkb import loads
#from shapely import wkt
from shapely.geometry import Point

# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Imported Libraries')



Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Imported Libraries


# Introduction

In this presentation we will define a problem to solve using Data Science techniques managing and processing data from different sources, including data pulled using an API, in this case, from Foursquare.com

The problem selected for this example is to determine best places to open a Coffee Shop in the city of Buenos Aires in the proximity of subway stations. 

There are a high number of variables to consider in such decision, but for the example of this practice, we will base our analysis in the data sets described below, leaving other important data, measurable or not, outside of the scope of the example.

# Problem Description

The problem selected is based in the city of Buenos Aires, in particular near the subway stations that concentrates a high number of people traveling daily across the city. 
Considering Coffee Shops are particularly popular in Argentina as meeting points for joining with friends, coworkers, and also for business, we will analyze different sets of data to determine best possible new place to open a new Coffee Shop. 


# Data Section

The data sets used along this study are:

From the Government of the city of Buenos Aires, official information:

1. Neighborhood information, with geographical location
1. Population for each Neighborhood from same source, but based on 2010 official data
1. Subway stations information, including geographical location and line
1. Premetro stations information (additional stations complimenting the subway network)
1. Traffic information of the subway stations

From Foursquare.com

1. Exploration of all Venues near subways stations
1. Exploration of specific categories of Coffee Shops near subway stations


### Buenos Aires city Neigborhood information with location data

In [3]:
borough=pd.read_csv('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.csv')
borough.head()

,WKT,barrio,comuna,perimetro,area
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15,7724.852955,3.115707e+06
1,"POLYGON ((-58.4655768128541 -34.5965577078058,...",PATERNAL,15,7087.513295,2.229829e+06
2,"POLYGON ((-58.4237529813037 -34.5978273383243,...",VILLA CRESPO,15,8131.857075,3.615978e+06
3,"POLYGON ((-58.4946097568899 -34.6148652395239,...",VILLA DEL PARQUE,11,7705.389797,3.399596e+06
4,"POLYGON ((-58.4128700313089 -34.6141162515854,...",ALMAGRO,5,8537.901368,4.050752e+06


In [4]:
borough=borough.rename(columns={'barrio':'Borough'}).sort_values(by=['Borough']).reset_index(drop=True) #rename column and sort
borough.head()

,WKT,Borough,comuna,perimetro,area
0,"POLYGON ((-58.4771156675186 -34.5951149914833,...",AGRONOMIA,15,6556.167772,2.122169e+06
1,"POLYGON ((-58.4128700313089 -34.6141162515854,...",ALMAGRO,5,8537.901368,4.050752e+06
2,"POLYGON ((-58.4119188098038 -34.5980030767748,...",BALVANERA,3,8375.821811,4.342280e+06
3,"POLYGON ((-58.3703353711449 -34.6329258371189,...",BARRACAS,4,12789.791771,7.953453e+06
4,"POLYGON ((-58.4505669109009 -34.5356104340406,...",BELGRANO,13,20609.775397,7.999240e+06


### Buenos Aires Population by Borough Dataset

In [5]:
population=pd.read_csv('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/caba_pob_barrios_2010.csv')
population=population.rename(columns={'BARRIO':'Borough', 'POBLACION':'Population'}) #rename column
population.head()

,Borough,Population
0,AGRONOMIA,13912
1,ALMAGRO,131699
2,BALVANERA,138926
3,BARRACAS,89452
4,BELGRANO,126267


In [6]:
#Join both Borough datasets together
borough=borough.join(population.set_index('Borough'), on='Borough')

In [7]:
print(borough.shape)
borough.head()

(48, 6)


,WKT,Borough,comuna,perimetro,area,Population
0,"POLYGON ((-58.4771156675186 -34.5951149914833,...",AGRONOMIA,15,6556.167772,2.122169e+06,13912
1,"POLYGON ((-58.4128700313089 -34.6141162515854,...",ALMAGRO,5,8537.901368,4.050752e+06,131699
2,"POLYGON ((-58.4119188098038 -34.5980030767748,...",BALVANERA,3,8375.821811,4.342280e+06,138926
3,"POLYGON ((-58.3703353711449 -34.6329258371189,...",BARRACAS,4,12789.791771,7.953453e+06,89452
4,"POLYGON ((-58.4505669109009 -34.5356104340406,...",BELGRANO,13,20609.775397,7.999240e+06,126267


### Subway Information dataset

In [8]:
#Read subway stations names with geo locations
sub=pd.read_csv('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/subte-estaciones/estaciones-de-subte.csv')
print(sub.shape)
sub.head()

(90, 5)


,long,lat,id,estacion,linea
0,-58.398928,-34.635750,1.0,CASEROS,H
1,-58.400970,-34.629376,2.0,INCLAN - MEZQUITA AL AHMAD,H
2,-58.402323,-34.623092,3.0,HUMBERTO 1°,H
3,-58.404732,-34.615242,4.0,VENEZUELA,H
4,-58.406036,-34.608935,5.0,ONCE - 30 DE DICIEMBRE,H


In [9]:
sub=sub.rename(columns={'estacion':'Station Name', 'linea':'Station Line'}) #rename column
sub=sub.drop(['id'],axis=1)
print(sub.shape)
sub.head()

(90, 4)


,long,lat,Station Name,Station Line
0,-58.398928,-34.635750,CASEROS,H
1,-58.400970,-34.629376,INCLAN - MEZQUITA AL AHMAD,H
2,-58.402323,-34.623092,HUMBERTO 1°,H
3,-58.404732,-34.615242,VENEZUELA,H
4,-58.406036,-34.608935,ONCE - 30 DE DICIEMBRE,H


### Premetro Data set
Additional metro stations

In [10]:
#Read the external extension to the subway lines called "Premetro"
sub_pre=pd.read_csv('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/premetro/estaciones-premetro.csv')
sub_pre=sub_pre.drop(['id'],axis=1)
sub_pre=sub_pre.rename(columns={'nombre':'Station Name', 'linea':'Station Line'}) #rename column
print(sub_pre.shape)
sub_pre.head()

(18, 4)


,long,lat,Station Line,Station Name
0,-58.461491,-34.643757,PREMETRO,INTENDENTE SAGUIER
1,-58.456415,-34.648601,PREMETRO,BALBASTRO (Cementerio de Flores)
2,-58.450047,-34.659499,PREMETRO,ANA MARÍA JANER
3,-58.446607,-34.662225,PREMETRO,FERNANDEZ DE LA CRUZ
4,-58.448513,-34.665368,PREMETRO,PTE. ILLIA (Lacarra)


In [11]:
#Join subway and premetro information
sub= pd.concat([sub,sub_pre], sort=False).sort_values('Station Name').reset_index(drop=True)
print(sub.shape)
sub.head()

(108, 4)


,long,lat,Station Name,Station Line
0,-58.380574,-34.604245,9 DE JULIO,D
1,-58.436429,-34.618280,ACOYTE,A
2,-58.407161,-34.591628,AGÜERO,D
3,-58.401208,-34.609834,ALBERTI,A
4,-58.420962,-34.603165,ALMAGRO - MEDRANO,B


### Stations Traffic information
YTD 2019 from http://cdn.buenosaires.gob.ar/datosabiertos/datasets/subte-viajes-molinetes/molinetes-2019.zip

In [12]:
#Read uncompressed dataset already loaded
sub_use=pd.read_csv(r'datahistorica082019.csv')
sub_use.head(15)

,periodo,fecha,desde,hasta,linea,molinete,estacion,pax_pagos,pax_pases_pagos,pax_franq,total
0,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_Lima_N_Turn02,Lima,1.0,0.0,0.0,1.0
1,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_Loria_N_Turn03,Loria,3.0,0.0,0.0,3.0
2,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_Miserere_Q_HALL_Turn01,Plaza Miserere,3.0,0.0,0.0,3.0
3,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_Miserere_S_Turn01,Plaza Miserere,6.0,0.0,0.0,6.0
4,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_Miserere_S_Turn03,Plaza Miserere,10.0,0.0,0.0,10.0
5,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_Peru_S_Turn03,Peru,2.0,0.0,0.0,2.0
6,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_PJunta_S_Turn04,Primera Junta,9.0,0.0,0.0,9.0
7,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_SanPedrito_Oeste_Turn05,San Pedrito,2.0,0.0,1.0,3.0
8,201901,2019-01-01,08:00:00,08:15:00,LineaA,LineaA_SnzPena_N_Turn01,Saenz PeÃÂ±a,1.0,0.0,0.0,1.0
9,201901,2019-01-01,08:00:00,08:15:00,LineaB,LineaB_Alem_N_Turn03,Leandro N. Alem,1.0,0.0,0.0,1.0


In [13]:
#Discard not needed columns
sub_use=sub_use.drop(['desde','hasta','pax_pagos','pax_pases_pagos','pax_franq'],1)
print('Subway 2019 passengers records dataset: ',sub_use.shape)

Subway 2019 passengers records dataset:  (8300108, 6)


In [14]:
sub_use.tail()

,periodo,fecha,linea,molinete,estacion,total
8300103,201908,2019-08-31,LineaD,LineaD_9Julio_S_Turn02,9 de julio,4.0
8300104,201908,2019-08-31,LineaD,LineaD_Pueyrredon_Turn01,Pueyrredon.D,1.0
8300105,201908,2019-08-31,LineaH,LineaH_Once_Norte_Turn04,Once,1.0
8300106,201908,2019-08-31,LineaH,LineaH_Once_Sur_Turn02,Once,1.0
8300107,201908,2019-08-31,LineaH,LineaH_Venezuela_Sur_Turn01,Venezuela,2.0


In [15]:
start=sub_use.iloc[0,1]
end=sub_use.iloc[-1,1]
print('Subway passengers initial date: ',start,' final date measured: ',end)

sub_use.groupby(['estacion','linea']).sum()[['total']]

Subway passengers initial date:  2019-01-01  final date measured:  2019-08-31


,,total
estacion,linea,
9 de julio,LineaD,1832363.0
Acoyte,LineaA,2891038.0
AgÃÂ¼ero,LineaD,450586.0
AgÃ¼ero,LineaD,1160895.0
Agüero,LineaD,630504.0
Alberti,LineaA,958847.0
Angel Gallardo,LineaB,3048723.0
Avenida La Plata,LineaE,984719.0
Avenida de Mayo,LineaC,1104846.0


### Foursquare Venues Explore Function

In [16]:
CLIENT_ID = 'JK4SBLGN3YU1DKHQS40VSJ2RUTBDQ0BF0VE2X2J0E2HSDR3M' # your Foursquare ID
CLIENT_SECRET = 'QXOOOJBSDA10TTVT50BRHURLN2NEBGMZEKAWC3XPHPS4XD2M' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JK4SBLGN3YU1DKHQS40VSJ2RUTBDQ0BF0VE2X2J0E2HSDR3M
CLIENT_SECRET:QXOOOJBSDA10TTVT50BRHURLN2NEBGMZEKAWC3XPHPS4XD2M


In [17]:
def getNearbyVenues(names, line, latitudes, longitudes, radius=800):
    LIMIT = 200 # limit of number of venues returned by Foursquare API
    radius = 800 # define radius
    venues_list=[]
    for name, line, lat, lng in zip(names, line, latitudes, longitudes):
         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            line,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['location']['distance'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station Line',
                  'Station Name',
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Distance',
                  'Venue Category']
    return(nearby_venues)


In [18]:
LIMIT = 30000 # limit of number of venues returned by Foursquare API
radius = 15000 # define radius
         # create the API request URL
latitudes='-34.619139'
longitudes='-58.443683'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius,
            LIMIT)

In [20]:
#Create dataframe with Venues of Buenos Aires
bsas_venues = getNearbyVenues(names=sub['Station Name'],
                                   line=sub['Station Line'],
                                   latitudes=sub['lat'],
                                   longitudes=sub['long'])

#To execute only once and avoid Foursquare limite save to disk
bsas_venues.to_csv (r'venues_sub.csv', index = None, header=True) 

In [21]:
#Readl from disk
bsas_venues=pd.read_csv(r'venues_sub.csv')
print(bsas_venues.shape)
bsas_venues.head()

(7246, 9)


,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,Obelisco - Plaza de la República,-34.603736,-58.381632,112,Monument / Landmark
1,D,9 DE JULIO,-34.604245,-58.380574,Teatro Gran Rex,-34.603237,-58.378928,188,Theater
2,D,9 DE JULIO,-34.604245,-58.380574,On Fit,-34.602386,-58.379985,213,Gym / Fitness Center
3,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop
4,D,9 DE JULIO,-34.604245,-58.380574,Tostado Café Club,-34.603494,-58.380793,85,Coffee Shop


#### Find and remove duplicated Venues

In [23]:
bsas_venues=bsas_venues.sort_values(by=['Station Name','Venue','Venue Latitude','Venue Longitude','Venue Distance']).reset_index(drop=True)
print(bsas_venues.shape)
bsas_venues.head(10)

(7246, 9)


,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,180 Burger Bar,-34.604848,-58.380574,67,Burger Joint
1,D,9 DE JULIO,-34.604245,-58.380574,474 Buenos Aires Hotel,-34.602514,-58.374013,631,Hotel
2,D,9 DE JULIO,-34.604245,-58.380574,725 Continental Hotel,-34.606234,-58.376724,416,Hotel
3,D,9 DE JULIO,-34.604245,-58.380574,Alimac,-34.601557,-58.374829,605,Video Game Store
4,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop
5,D,9 DE JULIO,-34.604245,-58.380574,Arcangel,-34.607182,-58.379870,333,Restaurant
6,D,9 DE JULIO,-34.604245,-58.380574,Asador La Estancia,-34.602488,-58.380096,200,Argentinian Restaurant
7,D,9 DE JULIO,-34.604245,-58.380574,BIGG CrossFit Downtown,-34.604621,-58.376212,401,Gym / Fitness Center
8,D,9 DE JULIO,-34.604245,-58.380574,Banchero,-34.603985,-58.385112,416,Pizza Place
9,D,9 DE JULIO,-34.604245,-58.380574,Bar The Bar,-34.608303,-58.379013,473,Restaurant


In [24]:
#Remove duplicated keeping closest Venue to the Station
bsas_venues=bsas_venues.drop_duplicates(subset=['Venue','Venue Latitude','Venue Longitude'], keep='first').reset_index(drop=True)
print(bsas_venues.shape)
bsas_venues.head(10)

(2804, 9)


,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,180 Burger Bar,-34.604848,-58.380574,67,Burger Joint
1,D,9 DE JULIO,-34.604245,-58.380574,474 Buenos Aires Hotel,-34.602514,-58.374013,631,Hotel
2,D,9 DE JULIO,-34.604245,-58.380574,725 Continental Hotel,-34.606234,-58.376724,416,Hotel
3,D,9 DE JULIO,-34.604245,-58.380574,Alimac,-34.601557,-58.374829,605,Video Game Store
4,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop
5,D,9 DE JULIO,-34.604245,-58.380574,Arcangel,-34.607182,-58.379870,333,Restaurant
6,D,9 DE JULIO,-34.604245,-58.380574,Asador La Estancia,-34.602488,-58.380096,200,Argentinian Restaurant
7,D,9 DE JULIO,-34.604245,-58.380574,BIGG CrossFit Downtown,-34.604621,-58.376212,401,Gym / Fitness Center
8,D,9 DE JULIO,-34.604245,-58.380574,Banchero,-34.603985,-58.385112,416,Pizza Place
9,D,9 DE JULIO,-34.604245,-58.380574,Bar The Bar,-34.608303,-58.379013,473,Restaurant


### Add Borough to each Venue

In [25]:
bsas_venues['Borough'] =  'NaN'
bsas_venues.head(5)

,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category,Borough
0,D,9 DE JULIO,-34.604245,-58.380574,180 Burger Bar,-34.604848,-58.380574,67,Burger Joint,NaN
1,D,9 DE JULIO,-34.604245,-58.380574,474 Buenos Aires Hotel,-34.602514,-58.374013,631,Hotel,NaN
2,D,9 DE JULIO,-34.604245,-58.380574,725 Continental Hotel,-34.606234,-58.376724,416,Hotel,NaN
3,D,9 DE JULIO,-34.604245,-58.380574,Alimac,-34.601557,-58.374829,605,Video Game Store,NaN
4,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop,NaN


In [26]:
for i in range(len(bsas_venues)):
    #print('Latitude', bsas_venues.iloc[i,5])
    #print('Longitude', bsas_venues.iloc[i,6])
    point=Point(bsas_venues.iloc[i,6],bsas_venues.iloc[i,5])
    for b in range(len(borough)):
        poly = shapely.wkt.loads(borough.iloc[b,0])
        if (poly.contains(point)):
            bsas_venues.iloc[i,9]=borough.iloc[b,1]
bsas_venues.head()

,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category,Borough
0,D,9 DE JULIO,-34.604245,-58.380574,180 Burger Bar,-34.604848,-58.380574,67,Burger Joint,SAN NICOLAS
1,D,9 DE JULIO,-34.604245,-58.380574,474 Buenos Aires Hotel,-34.602514,-58.374013,631,Hotel,SAN NICOLAS
2,D,9 DE JULIO,-34.604245,-58.380574,725 Continental Hotel,-34.606234,-58.376724,416,Hotel,SAN NICOLAS
3,D,9 DE JULIO,-34.604245,-58.380574,Alimac,-34.601557,-58.374829,605,Video Game Store,SAN NICOLAS
4,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop,SAN NICOLAS


In [28]:
print(bsas_venues.shape)

(2804, 10)


In [29]:
bsas_venues.head()

,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category,Borough
0,D,9 DE JULIO,-34.604245,-58.380574,180 Burger Bar,-34.604848,-58.380574,67,Burger Joint,SAN NICOLAS
1,D,9 DE JULIO,-34.604245,-58.380574,474 Buenos Aires Hotel,-34.602514,-58.374013,631,Hotel,SAN NICOLAS
2,D,9 DE JULIO,-34.604245,-58.380574,725 Continental Hotel,-34.606234,-58.376724,416,Hotel,SAN NICOLAS
3,D,9 DE JULIO,-34.604245,-58.380574,Alimac,-34.601557,-58.374829,605,Video Game Store,SAN NICOLAS
4,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop,SAN NICOLAS


### Explore by Category to get only Cafes of Coffe Shops
Since Venues per station reached Foursqueare maximum hits, will develop new explore query to bring only Cafe or Coffe Shopps Venues for a more detailed set of data

In [33]:
#Define new function with category explore for Caffe or Coffe Shops
def getNearbyVenuesCat(names, line, latitudes, longitudes):
    LIMIT = 200 # limit of number of venues returned by Foursquare API
    radius = 800 # define radius
    venues_list=[]
    for name, line, lat, lng in zip(names, line, latitudes, longitudes):
         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId=4bf58dd8d48988d1e0931735&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            line,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['location']['distance'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station Line',
                  'Station Name',
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Distance',
                  'Venue Category']
    return(nearby_venues)

In [34]:
#Create dataframe with Venues category of Caffe or Coffe Shops in Buenos Aires near subway stations
bsas_cafes = getNearbyVenuesCat(names=sub['Station Name'],
                                   line=sub['Station Line'],
                                   latitudes=sub['lat'],
                                   longitudes=sub['long']
                                   )
bsas_cafes.to_csv (r'cafes.csv', index = None, header=True)

In [35]:
bsas_cafes=pd.read_csv(r'cafes.csv')
print(bsas_cafes.shape)
bsas_cafes.head()

(3699, 9)


,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Coffee Shop
1,D,9 DE JULIO,-34.604245,-58.380574,Tostado Café Club,-34.603494,-58.380793,85,Coffee Shop
2,D,9 DE JULIO,-34.604245,-58.380574,Starbucks,-34.603542,-58.378715,187,Coffee Shop
3,D,9 DE JULIO,-34.604245,-58.380574,NEGRO. Cueva de café,-34.600848,-58.379612,388,Coffee Shop
4,D,9 DE JULIO,-34.604245,-58.380574,Starbucks,-34.604619,-58.381821,121,Coffee Shop


In [36]:
bsas_cafes["Venue Category"].replace("Coffee Shop","Café",inplace=True)
bsas_cafes.head()

,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Café
1,D,9 DE JULIO,-34.604245,-58.380574,Tostado Café Club,-34.603494,-58.380793,85,Café
2,D,9 DE JULIO,-34.604245,-58.380574,Starbucks,-34.603542,-58.378715,187,Café
3,D,9 DE JULIO,-34.604245,-58.380574,NEGRO. Cueva de café,-34.600848,-58.379612,388,Café
4,D,9 DE JULIO,-34.604245,-58.380574,Starbucks,-34.604619,-58.381821,121,Café


Following table shows the venues reached the limit of 100 venues from foursquare

In [37]:
#Check quantity if venues
bsas_cafes.set_index(["Station Name", "Station Line"]).count(level="Station Name").head()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
Station Name,,,,,,,
9 DE JULIO,100,100,100,100,100,100,100
ACOYTE,30,30,30,30,30,30,30
AGÜERO,58,58,58,58,58,58,58
ALBERTI,23,23,23,23,23,23,23
ALMAGRO - MEDRANO,27,27,27,27,27,27,27


but still need to cleanup duplicates so lets

### Eliminate Venues duplicates

In [38]:
bsas_cafes=bsas_cafes.sort_values(by=['Station Name','Venue','Venue Latitude','Venue Longitude','Venue Distance']).reset_index(drop=True)
print(bsas_cafes.shape)
bsas_cafes.head()

(3699, 9)


,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Café
1,D,9 DE JULIO,-34.604245,-58.380574,Alma Café,-34.604919,-58.382356,179,Café
2,D,9 DE JULIO,-34.604245,-58.380574,Anamora,-34.602653,-58.385472,482,Café
3,D,9 DE JULIO,-34.604245,-58.380574,Bar Dado,-34.605032,-58.387940,680,Café
4,D,9 DE JULIO,-34.604245,-58.380574,Bernardo Café,-34.609498,-58.380731,584,Café


In [39]:
#Remove duplicated keeping closest Venue to the Station
bsas_cafes=bsas_cafes.drop_duplicates(subset=['Venue','Venue Latitude','Venue Longitude'], keep='first').reset_index(drop=True)
print(bsas_cafes.shape)
bsas_cafes.head(5)

(893, 9)


,Station Line,Station Name,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,D,9 DE JULIO,-34.604245,-58.380574,All Saints Cafe,-34.603559,-58.378070,241,Café
1,D,9 DE JULIO,-34.604245,-58.380574,Alma Café,-34.604919,-58.382356,179,Café
2,D,9 DE JULIO,-34.604245,-58.380574,Anamora,-34.602653,-58.385472,482,Café
3,D,9 DE JULIO,-34.604245,-58.380574,Bar Dado,-34.605032,-58.387940,680,Café
4,D,9 DE JULIO,-34.604245,-58.380574,Bernardo Café,-34.609498,-58.380731,584,Café


In [40]:
bsas_cafes.set_index(["Station Name", "Station Line"]).count(level="Station Name").head()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
Station Name,,,,,,,
9 DE JULIO,100,100,100,100,100,100,100
ACOYTE,30,30,30,30,30,30,30
AGÜERO,58,58,58,58,58,58,58
ALBERTI,23,23,23,23,23,23,23
ALMAGRO - MEDRANO,27,27,27,27,27,27,27


In [41]:
bsas_cafes[['Station Name','Station Line','Venue']].groupby(['Station Name','Station Line']).count().sort_values('Venue',ascending=False).head()

,,Venue
Station Name,Station Line,
9 DE JULIO,D,100
CALLAO,D,80
AGÜERO,D,58
CATALINAS,E,55
AV. DE MAYO,C,40


-------------
------------
------------
